<a href="https://colab.research.google.com/github/onedayzero/food-sales-predictions/blob/main/food_sales_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Sales Predictions
-Clint Atterberry

-25 May 2022

# Instructions

1) How many rows and columns?

2) What are the datatypes of each variable?

3) Are there duplicates? If so, drop any duplicates.

4) Identify missing values.

5) Decide on how to address the missing values and do it! (This requires your judgement, so explain your choice).

6) Confirm that there are no missing values after addressing them.

7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent) 

8) For any numerical columns, obtain the summary statistics of each (min, max, mean)

## Mount the drive, import libraries, and import data

-We will use the following lines:

```python
# Get access to the drive where the data is
from google.colab import drive
drive.mount ('/content/drive')

# The library we will be using
import pandas as pd

# The csv file from the drive
df = pd.read_csv('/content/drive/MyDrive/Coding Dojo - Data Science Course/01 Week 1: Python for Data Science/sales_predictions.csv')

# To get a glance at the columns and the first 5 rows
df.head()
```

In [1]:
# Mount the drive
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
# Import Library pandas as pd
import pandas as pd

In [3]:
# Get csv data from link for assignment
df = pd.read_csv('/content/drive/MyDrive/Coding Dojo - Data Science Course/01 Week 1: Python for Data Science/sales_predictions.csv')

In [4]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


## 1) How many rows and columns?

- We will use the following lines:

```python
# We will find out the shape (rows, columns) by using .shape
df.shape
```

- There are 8523 rows and 12 columns.

In [ ]:
# This will display the (rows, columns) of the data frame
df.shape

(8523, 12)

## 2) What are the datatypes of each variable?

- We will use the following lines:

```python
# This will display the data types of all variables
df.dtypes

# object - strings
# float64 - floats
# int64 - integers
```

- objects (strings) - 'Item_Identifier', 'Item_Fat_Content', 'Item_Type', 'Outlet_Identifier', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_type'

- float64 (floats) - 'Item_Weight', 'Item_Visibility', 'Item_MRP', 'Item_Outlet_Sales'

- int64 (integers) - 'Outlet_Establishment_Year'

In [ ]:
# This will display the data types of the variables
df.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

## 3) Are there duplicates? If so, drop any duplicates.

- We will use the following lines:

```python
# This will display the duplicated rows
df.duplicated().value_counts()

# True - it is a duplicated row
# False - it is not a duplicated row
```

- There are no duplicate rows; therefore, there are no rows to drop.

In [ ]:
# This will display the counts of the duplicated rows
df.duplicated().value_counts()

False    8523
dtype: int64

## 4) Identify missing values.

- We will use the following lines:

```python
# This will display the missing values for each column
df.isna().sum()
```

- There are 2 rows that appear to have missing values:

- Item_Weight is missing 1463 values

- Outlet_Size is missing 2410 values

- There may be more missing values in other columns but they may have a placeholder (like 'missing') in its place in the column. Item_Visibility has some values that are 0, so those may be missing values.

In [ ]:
df.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

## 5) Decide on how to address the missing values and do it! (This requires your judgement, so explain your choice).

- We will start with the following lines:

```python
# First lets find out some summary statistics of our columns with missing values

# Using .describe() because it is float type data
df.['Item_Weight'].describe()
# Using .value_counts() because it is string type data
df.['Outlet_Size'].value_counts()

# Next we will make a decision for each column considering multiple factors
# The factors I will consider are:
# - What percent of the data is missing (If a relatively small amount of the data is missing, then I will consider deleting those rows)
# - What is the min, max, median, mean and std deviation of the data (Are the values close together or are they spread out, if close together I might place the mean values in place of those missing values)
# - In context, is the missing data important to the rest of the data set (would someone care or even notice if those data points were missing?)
```

In [ ]:
# We have 82.8% of the data
df['Item_Weight'].describe()

# With over 7000 data values and 1600 missing values
# There are no outliers in the data set when considering both the mean and the median
# The mean + 2 standard deviations from the mean captures no data points
# The median +- 1.5 * the IQR produces fences that are beyond both the min and max

# I believe the best course of action is to use the mean and filling in the gaps of the missing data points
# I chose the mean over the median because the mean is very close to the middle of the max and the min (~12.9)

count    7060.000000
mean       12.857645
std         4.643456
min         4.555000
25%         8.773750
50%        12.600000
75%        16.850000
max        21.350000
Name: Item_Weight, dtype: float64

In [ ]:
# We have 71.7% of the data
df['Outlet_Size'].value_counts()

# With nearly 30% of the values missing the size of the outlet
# The counts are not very close considering ~45% of the known data is Medium, ~39% of the data is known small, and the rest (~15%) is High
# One might consider filling in the missing values with the mode, Medium
# I believe that would be filling in too much data, nearly 30%, with one value will not be the best representation
# Due to the size of the missing data and that it is categorical, either deleting those rows or filling in the data with "Missing" would be the best course of action
# There is a other, similar columns like 'Outlet_Location_Type'and 'Outlet_Type' that probably describes the data in similar way

# I believe the best course of action at this time is to remove the entire column
# I am basing this decision on the fact that there are similar columns to 'Outlet_Size' like 'Outlet_Type' and 'Outlet_Location_Type'

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

## Checking in:

- We will drop the 'Outlet_Size' column with the following line:
```python
df.drop(columns='Outlet_Size', inplace=True)
```

- We will fill in the missing values in 'Item_Weight' with the following lines:
```python
weight_mean = df['Item_Weight'].mean()
df['Item_Weight'].fillna(weight_mean, inplace=True)
```

In [ ]:
# We are going to start by getting rid of the column 'Outlet_Size'
df.drop(columns='Outlet_Size', inplace=True)

In [ ]:
# We are going to fill in the missing values with the mean value of the column

# Saving the mean as a variable 'weight_mean'
weight_mean = df['Item_Weight'].mean()

# Dropping the 'na' values with the 'weight_mean' variable
df['Item_Weight'].fillna(weight_mean, inplace=True)

## 6) Confirm that there are no missing values after addressing them.

- We will explore the sum of missing values from our data frame using:
```python
# This subs the missing values
df.isna().sum()
```

In [ ]:
# Finding the sum of missing values across the data frame
df.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

## Success!

- Our data frame now has no missing values

## 7) Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent)

- We will pull up all of the category names and verify the capitalization is consistent and there are no spaces using the following line:

```python
# This will show all of the columns, their index, a count of their values, and their data type
df.info()
```

In [30]:
# I will infestigate the columns made of strings

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [20]:
# Here are some inconsistencies I need to fix with Low Fat and Regular

df['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [29]:
# This will replace the LF and low fat to both Low Fat, and reg to Regular

df['Item_Fat_Content'].replace({'LF': 'Low Fat', 'reg': 'Regular', 'low fat': 'Low Fat'}, inplace=True)
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [31]:
# No changes need to be made

df['Item_Type'].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [32]:
# No changes need to be made

df['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [35]:
# No changes need to be made

df['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [37]:
# No changes need to be made

df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

## 8) For any numerical columns, obtain the summary statistics of each (min, max, mean)

- There numerical columns we will explore are:
1.   Item_Weight
2.   Item_Visibility
3.   Item_MRP
4.   Outlet_Establishment_Year
5.   Item_Outlet_Sales

- We will use the following method for each:

```python
# This provides the summary statistics
.describe()
```


In [ ]:
# Item_Weight summary statistics
df['Item_Weight'].describe()

count    8523.000000
mean       12.857645
std         4.226124
min         4.555000
25%         9.310000
50%        12.857645
75%        16.000000
max        21.350000
Name: Item_Weight, dtype: float64

## Summary Statistics for 'Item_Weight'
- min = 4.55
- max = 21.35
- mean = 12.86

In [ ]:
# Item_Visibility summary statistics
df['Item_Visibility'].describe()

count    8523.000000
mean        0.066132
std         0.051598
min         0.000000
25%         0.026989
50%         0.053931
75%         0.094585
max         0.328391
Name: Item_Visibility, dtype: float64

## Summary Statistics for 'Item_Visibility'
- min = 0
- max = 0.33
- mean = 0.07

In [ ]:
# Item_MRP summary statistics
df['Item_MRP'].describe()

count    8523.000000
mean      140.992782
std        62.275067
min        31.290000
25%        93.826500
50%       143.012800
75%       185.643700
max       266.888400
Name: Item_MRP, dtype: float64

## Summary Statistics for 'Item_MRP'
- min = 31.29
- max = 266.89
- mean = 140.99

In [ ]:
# Outlet_Establishment_Year summary statistics
df['Outlet_Establishment_Year'].describe()

count    8523.000000
mean     1997.831867
std         8.371760
min      1985.000000
25%      1987.000000
50%      1999.000000
75%      2004.000000
max      2009.000000
Name: Outlet_Establishment_Year, dtype: float64

## Summary Statistics for 'Outlet_Establishment_Year'
- min = 1985
- max = 2009
- mean = 1997

In [ ]:
# Item_Outlet_Sales summary statistics
df['Item_Outlet_Sales'].describe()

count     8523.000000
mean      2181.288914
std       1706.499616
min         33.290000
25%        834.247400
50%       1794.331000
75%       3101.296400
max      13086.964800
Name: Item_Outlet_Sales, dtype: float64

## Summary Statistics for 'Item_Outlet_Sales'
- min = 33.29
- max = 13,086.96
- mean = 2181.29